In [ ]:
-- select * from  dsc_dwd.dwd_wh_dsc_inventory_dtl_di 
-- where src = 'scale'
-- and data_source in ('scale_hpi', 'scale_michelin', 'scale_fas', 'scale_bose')
-- and inc_day in ('20211124', '20211117', '20211110', '20211103', 
-- '20211027', '20211020', '20211013','20211006')
-- and usage_flag = '1'

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import re
import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

print(os.listdir('data_down'))

['inv_7.csv']


In [42]:
%%time
df = pd.read_csv('./data_down/inv_7.csv', sep='\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
df = df.dropna(how = 'all', axis =1)
time_cols = pd.Series(df.columns)[pd.Series(df.columns).str.lower().str.findall('date|time').apply(len)>0]
df[time_cols] = df[time_cols].apply(lambda x: x.str.slice(0,10))
df9 = df 

Wall time: 7.9 s


In [43]:
def load_data(data_source):
    """
    load bose data;
    所有类型的qty都要加起来哦
    只选择有多个收货日期的货物
    """
    df = df9
    df = df[df['data_source'] == data_source]
    df ['recived_date'] = pd.to_datetime(df['recived_date'])
    df = df[['wms_company_name', 'wms_warehouse_id','sku_code', 'sku_name', 'sku_desc', 'location_zone',\
        'lock_codes', 'on_hand_qty', 'in_transit_qty','allocated_qty', 'shelf_days', 
        'recived_date','usage_flag', 'inc_day']]
    qty = pd.Series(df.columns).str.extract('(.+qty)').dropna(axis = 0)
    df['qty'] = df[qty[0]].sum(axis = 1).round(2)
    # bose_inv.drop_duplicates(subset = ['sku_code', 'recived_date', 'lock_codes', 'inc_day', 'qty'])
    # 没有重复的 目前看....aaa
    df = df.groupby(['recived_date', 'sku_code', 'lock_codes','inc_day', 'wms_warehouse_id'], dropna = False).agg(
    {
        'qty':sum,
        'location_zone': set
    }
    ).sort_values(['sku_code', 'recived_date']).reset_index()
    # 只选择有多个收货日期的货物
    filter0 = df.groupby(['sku_code'])['recived_date'].agg(
    {
        set
    }
        ).reset_index()

    filter0 = pd.DataFrame(filter0[filter0['set'].apply(len)> 1]['sku_code'].drop_duplicates())
    bose_inv = filter0.merge(df, on = ['sku_code'], how = 'inner')\
        .sort_values(['recived_date','sku_code', 'inc_day'])
    
    return bose_inv

bose_inv = load_data('scale_bose')

In [44]:
# # 只选择多次出库的货物
# filter = bose.groupby(['recived_date', 'sku_code'])['inc_day'].agg({set}).reset_index()
# filter = filter[filter['set'].apply(len)> 1][['recived_date','sku_code']].drop_duplicates();filter
bose_inv.head(4)

,sku_code,recived_date,lock_codes,inc_day,wms_warehouse_id,qty,location_zone
13036,722713-0010,2015-09-08,1000,20211006,BOSE_SH,13.0,{L-CE-1000}
13037,722713-0010,2015-09-08,1000,20211013,BOSE_SH,13.0,{L-CE-1000}
13038,722713-0010,2015-09-08,1000,20211020,BOSE_SH,13.0,{L-CE-1000}
13039,722713-0010,2015-09-08,1000,20211027,BOSE_SH,13.0,{L-CE-1000}


In [216]:
%%time
df0 = pd.DataFrame()
def snapshot():
    """
    pivot table. inc_day 快照 作为 cols
    添加标记.
    """
    global df0, bose_inv
    for i in tqdm(bose_inv['sku_code'].unique(), ascii= False, colour = 'green'):
        df_out = bose_inv[bose_inv['sku_code'] == i]\
            .pivot_table(columns=['inc_day'], index = 'recived_date', values=['qty']).reset_index()
        df_out['sku_code'] = i
        df0 = pd.concat([df0, df_out], axis = 0)
    try:
        df0.columns = df0.columns.get_level_values(level=1)
    except:
        pass
    df0.columns = list(df0.columns[0:8]) + ['received_date','sku']
    df0 = df0.sort_values(['sku', 'received_date'])
    df0['mark'] = 0
    # 这个吊东`西不知道为什么是反选的 
    df0['mark'] = df0['mark'].where(df0.iloc[:, 0:7].isna().all(axis = 1) == False, 'new')
    df0['mark'] = df0['mark'].where(~df0.iloc[:,7].isna() , 'clear')
    # fill na~
    df0 = df0.fillna(0)
    bose_inv = bose_inv.rename({'sku_code':'sku', 'recived_date':'received_date'}, axis = 1)\
        .reset_index(drop = True).drop(['inc_day', 'qty'], axis = 1)
    bose_inv = bose_inv.drop_duplicates(subset = ['sku', 'received_date', 'lock_codes'])
    df0 = df0.merge(bose_inv, on = ['sku','received_date'], how = 'left')

    # may lock
    df0['mark'] = df0['mark'].where(df0.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
snapshot()

Wall time: 1.02 ms


In [74]:
def err_part():
    """
    findout who are the naught peach.
    err 中干掉了 new 干掉了maylock 
    """
    df_err = df0[df0['mark'] != 'new']
    # 补充可能被锁的标记
    # df_err['mark'] = df_err['mark'].where(df_err.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
    # 干掉了maylock
    df_err = df_err[df_err['mark'] != 'may_lock'].sort_values(['sku', 'received_date'])
    df_err['change'] = df_err.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = df_err.groupby('sku').shift(1) 
    # q = []
    # for i in shift['mark']:
    #     if pd.isna(i):
    #         q.append('clear')
    #     elif i =='clear':
    #         q.append('clear')
    #     else:
    #         q.append('check')
    # shift['mark'] = q
    # shift['mark2'] = shift.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = shift[['mark','change']]
    shift.columns = ['lag_mark', 'lag_change']
    shift['lag_mark'] = shift['lag_mark'].where(~shift['lag_mark'].isna(), 'clear')
    df_err = pd.concat([df_err, shift], axis = 1)
    # df_wrong = df_err[(df_err['lag_mark'] != 'clear') & (df_err['change'] != 0)]
    # df_good  = df_err[(df_err['lag_mark'] != 'clear') | (df_err['change'] != 0)]
    return df_err
df_err = err_part()
    # shift['mark'] = ['clear' if (i == np.nan) 'clear'  elif (i =='clear') else 'check' for i in shift['mark']];shift

In [75]:
df_err.to_pickle('./data_up/df_err_bose.p')
df0.to_pickle('./data_up/df0_bose.p')
df_err_bose = pd.read_pickle('./data_up/df_err_bose.p')
df0_bose = pd.read_pickle('./data_up/df0_bose.p')
df0_bose = df0_bose[df0_bose['wms_warehouse_id'] == 'BOSE_SH']
df_err_bose = df_err_bose[df_err_bose['wms_warehouse_id'] == 'BOSE_SH']

In [196]:

def output(df_err, df0):
    dishes = list(df_err[(df_err['lag_mark'] != 'clear') & (df_err['change'] < 0)]['sku'].unique())
    return df0[df0['sku'].isin(dishes)]
def check(sku, df0):
    return df0[(df0['lock_codes'] == '1000') & (df0['sku'].isin(sku))].sort_values(['sku','received_date'])


In [197]:
view = output(df_err_bose,df0_bose)
# 1000全新 3000退货 4000破损产品-

In [202]:
def ou_level_lock_codes(lock_code_to_eliminate):
    # output(df_err_bose,df0_bose)['sku'].unique()
    select_none_lock  = pd.DataFrame(view.groupby('sku')['mark'].apply(list).astype(str).str.match('.+may')).reset_index()
    select_none_lock2 = pd.DataFrame(view.groupby('sku')['lock_codes'].apply(list).astype(str).str.match('.+'+lock_code_to_eliminate)).reset_index()
    bose_err_list = set(select_none_lock[~select_none_lock['mark']]['sku'].unique())
    bose_err_list2 = list(select_none_lock2[~select_none_lock2['lock_codes']]['sku'].unique())
    bose_err_list = list(bose_err_list.intersection(bose_err_list2))
    bose_err_list = list(set(bose_err_list))
    return bose_err_list

In [ ]:
bose_err_list = ou_level_lock_codes('4000|3000')

In [193]:
# bose_err_listd

In [208]:
bose_definite_wrong = check(bose_err_list, df0_bose)

=================================
# HP
=================================


In [218]:
%%time
del df0 ,bose_inv
bose_inv = load_data('scale_hpi');bose_inv
df0 = pd.DataFrame()
snapshot()

# snapshot()
df_err = err_part()

100%|██████████| 1483/1483 [00:31<00:00, 47.76it/s]


Wall time: 33.8 s


In [221]:
df_err.to_pickle('./data_up/df_err_hp.p')
df0.to_pickle('./data_up/df0_hp.p')
df_err_hp = pd.read_pickle('./data_up/df_err_hp.p')
df0_hp = pd.read_pickle('./data_up/df0_hp.p')


In [222]:
output(df_err_hp, df0_hp)

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
41,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-07-13,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
42,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-07-19,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
43,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-07-22,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
44,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-08-05,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
45,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-08-09,1VV21AA,clear,1004-34,HPI_SH,{L_Normal}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12138,10.0,10.0,4.0,4.0,4.0,4.0,4.0,0.0,2021-09-29,Z7Y79A,clear,1001-34,HPI_WH,{L_Normal}
12188,36.0,18.0,6.0,6.0,6.0,6.0,6.0,6.0,2021-08-31,Z9M07A,0,1001-34,HPI_WH,{L_Normal}
12189,36.0,18.0,6.0,6.0,6.0,6.0,6.0,6.0,2021-08-31,Z9M07A,0,1002-34,HPI_WH,{L_Normal}
12190,36.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-09-27,Z9M07A,clear,1001-34,HPI_WH,{L_Normal}


In [237]:
view = output(df_err_hp, df0_hp)
err_list_hp = ou_level_lock_codes('99')

In [238]:
err_list_hp

['W9052MC',
 '3YM17AA',
 'CF323AC',
 'W1108AD',
 'SU927A',
 'CZ706A',
 'CN628AA',
 'C9449A',
 'F6V27AA#AB2',
 'L0R06A',
 'SS582A',
 'T6N10AA',
 'Y1G23A',
 'G0Y77A',
 'W9212MC',
 '4YL17MC',
 'G3Q75A#AB2',
 'F5S41D#AB2',
 'CN046AA',
 'CF256A',
 'D9L63A#AB2',
 'CN047AA',
 '3WX02A',
 'ST976A',
 '5RC89A#AB2',
 'CR312AA#AB2',
 'CF256X',
 'J8030A',
 'M0J80AA',
 'C8765ZZ#AB2',
 'CF353A',
 'CN693AA',
 'W9191MC',
 'M0H55AA',
 'W2081A',
 'CF413A',
 'W9055MC',
 'G3Q61A#AB2',
 'Q2612A',
 'CF211A',
 'D8J07A',
 'SS823A',
 'NHXHC1823',
 'F9J70A',
 'NHXHC1821',
 'SS687A',
 'SS711A',
 'C9364ZZ#AB2',
 'D3Q23A',
 'CE401A',
 'W9053MC',
 'W2082A',
 'C2P25AA',
 'CF471XC',
 'CE278A',
 'L0S63AA',
 'X4D11AC',
 'T8W39B#AB2',
 'F6V20AA#AB2',
 'CF512A',
 'CF228A',
 'CF400X',
 'CF322AC',
 'F6V26AA#AB2',
 'Z7Y70A',
 'Z6Z11A#AB2',
 'CE411A',
 'C9352AA#007',
 'CC640ZZ',
 'CF325XC',
 '9RR49A',
 'L0R91AA',
 'W9102MC',
 '4ZA23A',
 'Z7Y79A',
 'M0K00AA',
 'W9048MC',
 'G3Q46A#AB2',
 'CH216A',
 'C1Q52A',
 'W9221MC',
 'CN622A

In [241]:
# df0_hp[df0_hp['sku'] == '1VV21AA']

In [242]:
def check(sku, df0):
    return df0[df0['sku'].isin(sku)]
hp_definite_wrong = check(err_list_hp, df0_hp)

# Z7Y79A 问一下
# 

=================================
# michelin
=================================


In [243]:
%%time
del df0 ,bose_inv
bose_inv = load_data('scale_michelin');bose_inv
df0 = pd.DataFrame()
snapshot()

# snapshot()
df_err = err_part()

100%|██████████| 1360/1360 [00:41<00:00, 32.47it/s]


Wall time: 44.2 s


In [ ]:
bose_inv

In [244]:
df_err.to_pickle('./data_up/df_err_mich.p')
df0.to_pickle('./data_up/df0_mich.p')
df_err_mich = pd.read_pickle('./data_up/df_err_mich.p')
df0_mich = pd.read_pickle('./data_up/df0_mich.p')


In [245]:
output(df0=df0_mich, df_err=df_err_mich)

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
35,32.0,32.0,32.0,32.0,32.0,32.0,32.0,32.0,2019-03-24,003718_102,may_lock,ONHAND_CN-1Y5,RT,{nan}
36,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,2020-02-23,003718_102,may_lock,ONHAND_CN-1Y5,RT,{nan}
37,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,2020-02-26,003718_102,may_lock,BLOCKED_TH-10C,RT,{nan}
38,7.0,7.0,7.0,7.0,7.0,7.0,7.0,11.0,2021-05-22,003718_102,0,Available,RT,{L_label}
39,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2021-06-22,003718_102,clear,RETURN_CN-1Y5,RT,{L_NC}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11492,17.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,2021-07-02,MATMICHELIN00584,0,Available,NaN,{nan}
11493,180.0,176.0,176.0,176.0,176.0,176.0,176.0,130.0,2021-08-14,MATMICHELIN00584,0,Available,NaN,{nan}
11494,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2021-10-12,MATMICHELIN00584,0,RETURN_IM-1IM,NaN,{nan}
11506,16.0,4.0,3.0,79.0,79.0,77.0,72.0,73.0,2021-07-03,MATMICHELIN00592,0,Available,NaN,{nan}


In [246]:
df0

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
0,15.0,14.0,18.0,10.0,10.0,10.0,10.0,10.0,2021-09-14,000219_111,0,Available,RT,{L_label}
1,96.0,96.0,96.0,96.0,96.0,96.0,96.0,96.0,2021-09-20,000219_111,may_lock,Available,RT,{L_label}
2,0.0,0.0,25.0,25.0,21.0,21.0,21.0,21.0,2021-10-15,000219_111,0,RETURN_FR-1F3,RT,{nan}
3,16.0,16.0,16.0,16.0,16.0,14.0,14.0,14.0,2020-12-02,000257_111,0,Available,RT,{L_label}
4,19.0,19.0,19.0,19.0,19.0,19.0,19.0,19.0,2020-12-11,000257_111,may_lock,RETURN_IM-1IM,RT,{L_label}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11503,16.0,13.0,13.0,13.0,13.0,13.0,12.0,4.0,2021-07-03,MATMICHELIN00591,0,Available,NaN,{nan}
11504,30.0,14.0,11.0,87.0,87.0,87.0,91.0,91.0,2021-09-02,MATMICHELIN00591,0,Available,NaN,{nan}
11505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2021-11-21,MATMICHELIN00591,new,RETURN_IM-1IM,NaN,{nan}
11506,16.0,4.0,3.0,79.0,79.0,77.0,72.0,73.0,2021-07-03,MATMICHELIN00592,0,Available,NaN,{nan}


In [ ]:
view = view = output(df0=df0_mich, df_err=df_err_mich)
err_list_hp = ou_level_lock_codes('gigiwang')

In [29]:
def check(sku, df0):
    return df0[(df0['lock_codes'] == 'Available') & (df0['sku'] == sku)]

check('MATMICHELIN00591',df0 = df0_mich)

# MATMICHELIN00591

# 同一时间 received 货物为什么会增加
# mich dot date?

,20211006,20211013,20211020,20211027,20211103,20211110,20211117,20211124,received_date,sku,mark,lock_codes,wms_warehouse_id,location_zone
11503,16.0,13.0,13.0,13.0,13.0,13.0,12.0,4.0,2021-07-03,MATMICHELIN00591,0,Available,NaN,{nan}
11504,30.0,14.0,11.0,87.0,87.0,87.0,91.0,91.0,2021-09-02,MATMICHELIN00591,0,Available,NaN,{nan}
